In [5]:
import torch
import torchvision
import numpy
import copy
import tqdm

from utils.model import LeNet5
from utils.lib import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark=True

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cuda


In [7]:
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=torchvision.transforms.ToTensor()
)

n_total_client = 5
n_data = 12000
communication_round = 2
epochs = 1
n_client = 2

model = LeNet5(28, 28, 1, 10)
idx_splited = idx_split(
    dataset=train_dataset,
    n_dataset=n_total_client,
    n_data_each_set=n_data
)
dataset_client = dict()
for i in range(n_total_client):
    dataset_client[i] = DealDataset(train_dataset, idx_splited[i])

server_model = copy.deepcopy(model)
tqdm_position = 0
for i in tqdm.tqdm(range(communication_round),
                   desc='communication_round',
                   position=tqdm_position):
    client = dict()
    client_param = dict()
    choicen_client = numpy.random.choice(
        range(n_total_client), n_client, replace=False)
    for j, k in enumerate(choicen_client):
        client[j] = train_model(
            model=server_model,
            dataset=dataset_client[k],
            device=device,
            epochs=epochs,
            tqdm_position=tqdm_position+1)
        client_param[j] = client[j].state_dict()
    server_model = Server(model=model, client_params=client_param).model

communication_round:   0%|          | 0/2 [00:00<?, ?it/s]

communication_round: 100%|██████████| 2/2 [00:05<00:00,  2.62s/it]


In [8]:
eval_model(client[0], test_dataset, device)
eval_model(client[1], test_dataset, device)
eval_model(server_model, test_dataset, device)

Test Accuracy: 93.89%
Test Accuracy: 94.04%
Test Accuracy: 94.29%
